In [237]:
#Ready for upload
using JuMP 
using Gurobi
using LightGraphs
using DataFrames
using CSV
using TimerOutputs
using Dates
myRun = Dates.format(now(), "HH:MM:SS")
gurobi_env = Gurobi.Env()
to = TimerOutput()
# myFile = "./SmallExample.jl"
# myFile = "./testing.jl"
myFile = "./myData1.jl"
# myInstance = 7
# myFile = "./Instances/testInstance_"*string(myInstance)*".jl"
include(myFile)
include("PartitionRules.jl")
include("functionGbound.jl")
include("functionHbound.jl")

# delta2= 0.01
# α = 0.1 
epsilon = 1e-3
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)

# If we want to add # in Gurobi, then we have to turn of Gurobi's own Cuts 
h1 = Model(() -> Gurobi.Optimizer(gurobi_env))
@variable(h1, 1 >= y_h[1:Len]>=0)
#@variable(h, q[1:Len]>=0)


#Setting constraint for start node
leaving = findall(edge[:,1].== origin)

@constraint(h1, sum(y_h[k] for k in leaving) == 1)
for i in all_nodes
    if i != destination && i != origin
        incoming = findall(edge[:,2].== i)
        leaving = findall(edge[:,1].== i)
        @constraint(h1, sum(-y_h[k] for k in leaving) + sum(y_h[k] for k in incoming) == 0)
    end
end

Academic license - for non-commercial use only


In [238]:
function what_arc(T, y, e)
    arc_type = -1 #type 0 = basic-SP, 1 = basic-nonSP, 2 = non-basic
    if T[e] == 1
        if y[e] == 1
            arc_type = 0
        else
            arc_type = 1
        end
    else
        arc_type = 2
    end
end

function classify_edges_costs(T,c)
    basic = []
    basic_cost = []
    i_basic = []
    nonbasic = []
    nonbasic_cost = []
    i_nonbasic = []
    
    for e = 1:Len
        #arc_type = what_arc(T,y,e)
        if T[e] != 1
            nonbasic = vcat(nonbasic, edge[e,:]')
            push!(i_nonbasic, e)
            push!(nonbasic_cost, c[e])
        else 
            basic = vcat(basic, edge[e,:]')
            push!(i_basic, e)
            push!(basic_cost, c[e])
        end
    end
    #We need to index back to the original edge values for reference purposes
    return basic, basic_cost, i_basic, nonbasic, nonbasic_cost, i_nonbasic
end


classify_edges_costs (generic function with 1 method)

In [239]:
function getSortedBasicArcList(pred, basic, i_basic)
#     succ = pred.*0
    sorted_basic = copy(basic)
    sorted_i_basic = zeros(Int64,length(i_basic))
#     println("pred = ", pred)
#     println("basic = ", basic)
#     println("i_basic = ", i_basic)
    
    index_sorting = length(i_basic)
#     println(index_sorting)
    S = [origin]
#     println("S = ",S)
    while isempty(S) == false
#         println("S = ",S)
        tailNode = S[1]
        deleteat!(S, 1)  
        for i = 1:length(pred)
            #origin vs destination
            if i != tailNode & pred[i] == tailNode
                u, v = tailNode, i
#                 println("(",u , ", ", v,")")
                myArc = transpose([u, v])
                temp_index = find(all(basic .== myArc,2))
                if length(temp_index) > 0
                    sorted_i_basic[index_sorting] = i_basic[temp_index[1]]
                    index_sorting = index_sorting - 1
                    push!(S,i)
                end
                
            end
        end        
    end
    return sorted_i_basic
#     println("sorted_i_basic = ", sorted_i_basic)
end

getSortedBasicArcList (generic function with 1 method)

In [240]:
function find_affected_node(T, pred, e, edge) #NEED TO CONSIDER CYCLES
    S = Int[]
    N = Int[]
    push!(S, edge[e,2])
    push!(N, edge[e,2])
    visit = collect(1:last_node)
    l = length(S)
    visit[edge[e,2]] = 0
    while l > 0 
        node = S[1]
        for i = 1:last_node 
            if pred[i] == node && visit[i] != 0
                push!(S, i)
                push!(N, i)
                visit[i] = 0
            end
            
        end
        S[1], S[length(S)] = S[length(S)], S[1]
        pop!(S)
        l = length(S)
    end
    return N
end

function Tree_basic(N, nonbasic, nonbasic_cost, label_temp, temp)
    delta_pos = []
    delta_pos_ind = []
    δ_pos_ind = 0
    val_pos_temp = 0
    delta_neg = []
    delta_neg_ind = []
    δ_neg_ind = 0
    val_neg_temp = 0
    
    if length(N) > 0
        ##println("nonbasic = ", nonbasic)
        for j in N
            head = find(nonbasic[:,2] .== j)
            if length(head) > 0
                ##println("head = ", head)
                for i in head
#                     println("Arc #", i)
#                     println("Arc name = ", nonbasic[i,:])
                    condd = nonbasic[i,1] in N
                    if condd == false
#                         println("π[",nonbasic[i,2],"] + Δ] - π[",nonbasic[i,1],"] - c_ij ")
#                         println("= ", label_temp[nonbasic[i,2]], " - ", label_temp[nonbasic[i,1]], " - ", nonbasic_cost[i])
                        val_pos_temp = label_temp[nonbasic[i,1]] + nonbasic_cost[i] - label_temp[nonbasic[i,2]]
#                         println("= ", val_pos_temp)
                        push!(delta_pos, val_pos_temp)
                        push!(delta_pos_ind, i)
                    end
                end
            end
        end
#         println("delta_pos = ", delta_pos)
        if length(delta_pos) > 0
            δ_pos_min = minimum(delta_pos)
            δ_pos_ind = find(delta_pos .== δ_pos_min)[1]
#             for i = 1:length(delta_pos)
#                 if i == 1
#                     δ_pos_min = delta_pos[1]
#                     δ_pos_ind = delta_pos_ind[1]
#                 end
#                 if delta_pos[i] < δ_pos_min
#                     δ_pos_min = delta_pos[i]
#                     δ_pos_ind = delta_pos_ind[i]
#                 end
#             end
#             temp = temp - δ_pos_min
        else
            δ_pos_min = -1.0
        end
#         print("δ_pos_min ", δ_pos_min)
        for j in N
            tail = find(nonbasic[:,1] .== j)
            if length(tail) > 0
                ##println("tail = ", tail)
                for i in tail
#                     println("Arc #", i)
#                     println("Arc name = ", nonbasic[i,:])
                    condd = nonbasic[i,2] in N
                    if condd == false
                        ##println("FALSE THEN")
                        val_neg_temp = label_temp[nonbasic[i,1]] + nonbasic_cost[i] - label_temp[nonbasic[i,2]]
#                         println("= ", val_neg_temp)
                        push!(delta_neg, val_neg_temp)
                        push!(delta_neg_ind, i)
                    end
                end
            end
        end
        
    end
    #IF DELTA DOESNT RETURN ANY VALUE AKA EMPTY: LET MIN DELTA = -1 THEN REMOVE IT LATER
    ###println("GOOD HERE")
#     println("delta_neg = ", delta_neg)
    if length(delta_neg) > 0
        δ_neg_min = minimum(delta_neg)
        δ_neg_ind = find(delta_neg .== δ_neg_min)[1]
#         for i = 1:length(delta_neg)
#             if i == 1
#                 δ_neg_min = delta_neg[1]
#                 δ_neg_ind = delta_neg_ind[1]
#             end
#             if delta_neg[i] < δ_neg_min
#                 δ_neg_min = delta_neg[i]
#                 δ_neg_ind = delta_neg_ind[i]
#             end
#         end
        temp = temp - δ_neg_min
    else
        δ_neg_min = -1.0
    end
#     print("δ_neg_min ", δ_neg_min)
#     print(temp)
    return δ_pos_min, δ_pos_ind, δ_neg_min, δ_neg_ind, temp
end

Tree_basic (generic function with 1 method)

In [241]:
function find_basic_Delta(myNode, Delta_Final_Pos, Delta_Final_Neg, sign, basic, i_basic, nonbasic, i_nonbasic)
    myNonTreeArcList = []
    myTreeArcList = []
    Delta_List = []
    delta = -1
#     println("myNode = ", myNode)
    if sign == "POS"
        myNonTreeArcList = find(all(nonbasic[:,2].== myNode,2))
        Delta_List = Delta_Final_Pos
    end
    if sign == "NEG"
        myNonTreeArcList = find(all(nonbasic[:,1].== myNode,2))
        Delta_List = Delta_Final_Neg
    end
    
    #NONBASIC ARCS
    if length(myNonTreeArcList) > 0
#         println("Nonbasic")
#         println("myNonTreeArcList = ", myNonTreeArcList)
        
        e = i_nonbasic[myNonTreeArcList[1]]
#         println("Arc ", e)
        delta = Delta_Final_Neg[e]
#         println("New delta = ", delta)
        for i = 2 : length(myNonTreeArcList)
            e = i_nonbasic[myNonTreeArcList[i]]
#             println("Arc ", e)
            if delta > Delta_Final_Neg[e] && Delta_Final_Neg[e] >= 0
                delta = Delta_Final_Neg[e]
#                 println("New delta = ", delta)
            end
        end
    end
    
    
    #BASIC ARCS
    myTreeArcList = find(all(basic[:,1].== myNode,2))
#     println("basic = ", basic)
#     println("myTreeArcList = ", myTreeArcList)
#     println("l = ",length(myTreeArcList))
    
#     println("myTreeArcList = ", myTreeArcList)
    if length(myTreeArcList) > 0
#         println("Basic")
        if delta == -1
            e = i_basic[myTreeArcList[1]]
            delta = Delta_List[e]
#             println("New delta = ", delta)
        end
        for i = 1 : length(myTreeArcList)
            e = i_basic[myTreeArcList[i]]
#             println("Arc ", e)
            if delta > Delta_List[e] && Delta_List[e] >= 0
                delta = Delta_List[e]
#                 println("New delta = ", delta)
            end
        end
    end
    return delta 
end


function find_SA_Delta(myNode, Delta_Final_Pos, Delta_Final_Neg, sign, basic, i_basic, nonbasic, i_nonbasic)
    myNonTreeArcList = []
    myTreeArcList = []
    Delta_List = []
    delta = maximum(Delta_Final_Neg)
    entering_arc = 0
    found = false
#     println("myNode = ", myNode)
    if sign == "POS"
        myNonTreeArcList = find(all(nonbasic[:,2].== myNode,2))
        Delta_List = Delta_Final_Pos
    end
    if sign == "NEG"
        myNonTreeArcList = find(all(nonbasic[:,1].== myNode,2))
        Delta_List = Delta_Final_Neg
    end
    
    #NONBASIC ARCS
    if length(myNonTreeArcList) > 0
#         println("Nonbasic")
#         println("myNonTreeArcList = ", myNonTreeArcList)
        
#         e = i_nonbasic[myNonTreeArcList[1]]
#         println("Arc ", e)
#         delta = Delta_Final_Neg[e]
#         println("New delta = ", delta)
        for i = 1 : length(myNonTreeArcList)
            e = i_nonbasic[myNonTreeArcList[i]]
#             println("Arc ", e, "Delta ", Delta_Final_Neg[e])
            if delta >= Delta_Final_Neg[e] && Delta_Final_Neg[e] >= 0
                delta = Delta_Final_Neg[e]
                entering_arc = e
#                 println("New delta = ", delta)
            end
        end
    end
    
    
    #BASIC ARCS
    myTreeArcList = find(all(basic[:,1].== myNode,2))
#     println("basic = ", basic)
#     println("myTreeArcList = ", myTreeArcList)
#     println("l = ",length(myTreeArcList))
    
#     println("myTreeArcList = ", myTreeArcList)
    if length(myTreeArcList) > 0
#         println("Basic")
        if delta == -1
#             e = i_basic[myTreeArcList[1]]
            delta = maximum(Delta_List)
#             println("New delta = ", delta)
        end
        for i = 1 : length(myTreeArcList)
            e = i_basic[myTreeArcList[i]]
#             println("Arc ", e, "Delta ", Delta_List[e])
            if delta >= Delta_List[e] && Delta_List[e] >= 0
                delta = Delta_List[e]
                entering_arc = e
#                 println("New delta = ", delta)
            end
        end
    end
#     println("entering arc inside function: ", entering_arc)
    return delta, entering_arc
end

function findNonBasicArc_Loop(label, T, c_g, e, path, y, pred, basic, i_basic)
    label_temp = copy(label)
    Δ = 0
    δ = []
    δ_ind = 0
    T_temp = copy(T)
    pred_temp = copy(pred)
    c_temp = copy(c_g)
    edge_num = copy(e)
    N = Int[]
    cur_edge = Int[]
    path_temp = copy(path)
    
    new_SP = false
    
    
    u = edge[e,1]
    v = edge[e,2]
    t = pred[v]
    e_temp = [t v]
    ###println(f,"(t v) = (",t, " ", v, ")")
    B = find(all(basic .== e_temp,2)) #find a tree edge to be replaced by nonbasic e
    i = B[1]
#     println("Entering arc = ", e )
#     println("Exiting arc = ", i)
    i = i_basic[i]
    val = label_temp[u] - label_temp[v] + c_temp[e]
    push!(δ, val)
    ###println(f,"δ = ", δ)
    δ_min = minimum(δ)
    c_temp[e] = c_temp[e] - δ_min
    Δ = Δ + δ_min
    pred_temp[v] = u
    ###println(f, "Tree before replace: ", T_temp)
    T_temp[e] = 1
    T_temp[i] = 0
    delta = -0.5
    while new_SP == false
        basic, basic_cost, i_basic, nonbasic, nonbasic_cost, i_nonbasic = classify_edges_costs(T_temp,c_temp)
        #loop = loop+1
        ###println("Loop ", loop)
        ##println( "Current T = ", T_temp)
        ##println( "Current y = ", y)
        ##println("Current label = ", label_temp)
        arc_type = what_arc(T_temp, y, e)
#         println("Double check: Arc ", e, " is now type ", arc_type)
        if arc_type == 0 #THIS STATEMENT IS WORKING
            new_SP = true
            delta = Δ
        end
        if arc_type == 1
            N = find_affected_node(T_temp, pred_temp, e, edge)
            ##println("N = ", N)
            δ_min, δ_ind, temp = SA_basic(N, nonbasic, nonbasic_cost, label_temp, c_temp[e])
            ##println("Δ = ", Δ)
            ##println("δ_min = ", δ_min)
            if δ_min < 0
                delta = -0.5 #δ_min
                new_SP = true
            else
                ##println("nonbasic arcs: ", nonbasic)
                ##println("index of arc moving into the basis: ", δ_ind)
                u = nonbasic[δ_ind, 1]
                v = nonbasic[δ_ind, 2]
                t = pred_temp[v]
                e_temp = [t v]
                ##println("Basic arc becoming nonbasic: ", e_temp)
                ##println("Nonbasic arc becoming basic: ", [u v])
                B = find(all(basic .== e_temp,2)) #find a tree edge to be replaced by nonbasic e
                i = B[1]
                i = i_basic[i]
                pred_temp[v] = nonbasic[δ_ind, 1]
                c_temp[e] = temp
                T_temp[i_nonbasic[δ_ind]] = 1
#                 println("Entering arc ", i_nonbasic[δ_ind])
#                 println("Exiting arc ", i)
                T_temp[i] = 0 ################
                Δ = Δ + δ_min
                delta = Δ
#                 println("current delta = ", delta)
                ##println("Pred = ", pred_temp)
            end
            
        end 
        #CHECK FOR LOOP:


        if y.*T_temp != y
            new_SP = true
        else
            loop = false
            if pred_temp[origin] != 0
                loop = true
            else
                v = edge[e,2]
                visit = collect(1:last_node)
                ##println("visit = ", visit)
                while v != origin && visit[v] != 0
                    ##println("v = ", v)
                    ##println("pred[v] = ", pred_temp[v] )
                    visit[v] = 0
                    v = pred_temp[v]

                    ##println("visit = ", visit)

                end

                if v != origin
                    loop = true
                    
                    ##println("HERE")
                end
            end
            if loop == true
                new_SP = true

            end
        end
        ##println("new_SP = ", new_SP)

    end

    return delta
end

function SA_basicSP(N, nonbasic, nonbasic_cost, label_temp, temp)
    delta = []
    delta_ind = []
    δ_ind = 0
    val_temp = 0
    if length(N) > 0
        ##println("nonbasic = ", nonbasic)
        for j in N
            head = find(nonbasic[:,2] .== j)
            if length(head) > 0
                ##println("head = ", head)
                for i in head
                    ##println("Arc #", i)
                    ##println("Arc name = ", nonbasic[i,:])
                    condd = nonbasic[i,1] in N
                    if condd == false
                        val_temp = label_temp[nonbasic[i,1]] + nonbasic_cost[i] - label_temp[nonbasic[i,2]]
                        push!(delta, val_temp)
                        push!(delta_ind, i)
                    end
                end
            end
        end
    end
    #IF DELTA DOESNT RETURN ANY VALUE AKA EMPTY: LET MIN DELTA = -1 THEN REMOVE IT LATER
    ###println("GOOD HERE")
    ##println("delta = ", delta)
    if length(delta) > 0
        for i = 1:length(delta)
            if i == 1
                δ_min = delta[1]
                δ_ind = delta_ind[1]
            end
            if delta[i] < δ_min
                δ_min = delta[i]
                δ_ind = delta_ind[i]
            end
        end
        temp = temp - δ_min
    else
        δ_min = -1.0
    end

    return δ_min, δ_ind, temp

end

function SA_basic(N, nonbasic, nonbasic_cost, label_temp, temp)
    delta = []
    delta_ind = []
    δ_ind = 0
    val_temp = 0
    if length(N) > 0
        ##println("nonbasic = ", nonbasic)
        for j in N
            tail = find(nonbasic[:,1] .== j)
            if length(tail) > 0
                ##println("tail = ", tail)
                for i in tail
                    ##println("Arc #", i)
                    ##println("Arc name = ", nonbasic[i,:])
                    condd = nonbasic[i,2] in N
                    if condd == false
                        ##println("FALSE THEN")
                        val_temp = label_temp[nonbasic[i,1]] + nonbasic_cost[i] - label_temp[nonbasic[i,2]]
                        push!(delta, val_temp)
                        push!(delta_ind, i)
                    end
                end
            end
        end
    end
    
    ##println("delta = ", delta)
    if length(delta) > 0
        ##println("DELTA > 0")
        for i = 1:length(delta)
            if i == 1
                δ_min = delta[1]
                δ_ind = delta_ind[1]
            end
            if delta[i] < δ_min
                δ_min = delta[i]
                δ_ind = delta_ind[i]
            end
        end
        temp = temp - δ_min
    else
        δ_min = -1.0
    end

    return δ_min, δ_ind, temp
end


function corner_g(y, pred, c_L, c_U, d, x_now, edge, origin, destination)
    c_corner = zeros(length(edge[:,1]))
    
    for i = 1:length(edge[:,1])
        if y[i] == 1
            c_corner[i] = c_U[i] + x_now[i]*d[i]
        else
            c_corner[i] = c_L[i] + x_now[i]*d[i]
        end
    end
    π = zeros(length(edge[:,1])) + sum(c_U[k] + x_now[k]*d[k] for k = 1:length(edge[:,1])) + 1
    u = origin
    y_index = find(y.==1)
    π[u] = 0
    S = [u]
    while u != destination
        found_node = false
        count = 0
        while found_node == false 
            count = count + 1
            i = y_index[count]
            if edge[i,1] == u
                found_node = true
                v = edge[i,2]
                π[v] = π[u] + c_corner[i]
                u = v
                push!(S, u)
            end
        end
    end
    B = copy(S) + 0
    
    opt = true
    while opt == true && length(B) > 0
        u = B[1]
        B[1], B[length(B)] = B[length(B)], B[1]
        pop!(B)
        myArcs = find(edge[:,1].==u)
        if length(myArcs) > 0
            for i in myArcs
                if y[i] != 1
                    v = edge[i,2]
                    if π[v] > π[u] + c_corner[i]
                        π[v] = π[u] + c_corner[i]
                        if length(find(B.==v)) == 0
                            push!(B, v)
                        end
                        if length(find(S.==v)) > 0
                            opt = false
                        end
                    end
                end
            end
        else
            succ = find(pred.==u)
            for i = 1:length(succ)
                if length(find(B.==succ[i])) > 0
                    push!(B, succ[i])
                end
            end
        end
    end
    return opt, c_corner
end

function Freeze1(K_not)
#     println("Inside function")
#     global α
#     println(df_Elim)
    K_α = []
    K_α_not = []
    K_not = []
    temp_df = sort(df_Elim, [:gU,:gL])
    δ3 = 1 #0.2
#     println("α = ", α)
#     if iter >= 38
#         println(temp_df)
#     end
    temp_p = 0
    
    i=0
    terminate_cond = false
    while temp_p < α
        i = i+1
        temp_p = temp_p + temp_df[i,:PROB]
    end
#     if temp_p <= 1
        max_g = temp_df[i,:gU]
#         println("max_g = ", max_g)
#         println("i = ", i)
        cell_i = 0
#         K_α = temp_df[!,:CELL][1:i] #Group of cells making up alpha
        if temp_p > α
            K_α = temp_df[!,:CELL][1:i]
            cell_i = temp_df[!,:CELL][i]
        else
            K_α = temp_df[!,:CELL][1:i]
        end 
        
        myRows = collect(i:nrow(temp_df))
        min_g = minimum(temp_df[!,:gL][myRows])
        
#     if iter >= 38
#         println("min_g = ", min_g)
#         println("max_g = ", max_g)
#     end
        temp_K = []
        if max_g - min_g > δ3
#             temp_K = filter(row -> row.gU > min_g, temp_df)[!,:CELL]  
        
#             println("temp_K = ", temp_K)
    #         println("max_gU = ", max_g)
    #         println("min_gL = ", min_g)
    #         println("i = ", i)
    #         min_g = minimum(temp_df[myRows,:gL])
    #         println("min_gU = ", min_g)
#             println("1")
#             println(myRows)
            temp_df = temp_df[myRows,:]
#             println(temp_df)
#             println("2")
    #         println(filter!(row -> row.gL >= max_g, temp_df))
    #         println(filter!(row -> row.gL < max_g, temp_df))
            K_not = filter(row -> row.gL >= max_g, temp_df)[!,:CELL]   
#             println("K_not = ", K_not)
    #         K_not = temp_df[!,:CELL] #Group of cells not in alpha that has gL > g_max 
            K_α_not = filter(row -> row.gL < max_g, temp_df)[!,:CELL] #Group of cells not in alpha
#             println("K_α_not = ", K_α_not)
        
#             if length(K_α_not) == 0#length(K_not) == nrow(df_Elim) - i
        else     
            terminate_cond = true
#             else
#                 K_α_not = vcat(K_α_not,temp_K)
#             end
        end
    
#     else
#         K_α = df_Elim[!,:CELL]
#         K_not = []
#     end
    
#     if length(K_not) == nrow(df_Elim) - i
#     println(temp_df)
#     push!(K_not,temp_df[!,:CELL])
    
#     println("K_α_not updated = ",K_α_not)
#     println("K_not length = ", length(K_not))
#     println("K_not weight total = ", sum(p[k] for k in K_not))
    return K_not, K_α, K_α_not, terminate_cond
    
end

Freeze1 (generic function with 1 method)

In [242]:
#MAIN PROGRAM:
df = DataFrame(CELL = Int[],ARCS_USED = Array[], LB = Array[], UB = Array[], PROB = Float64[], SP = Float64[])
df_Elim = DataFrame(CELL = Int[], gL = Float64[], gU = Float64[], PROB = Float64[])
df_gh = DataFrame(CELL = Int[],ARCS_USED = Array[], g = Float64[], h = Float64[], PROB = Float64[])
last_x = zeros(Len)

MP_obj = 0.0

push!(df, (1, yy, cL_orig, cU_orig, 1, SP_init))
push!(df_Elim, (1,0,0,1))
push!(df_gh, (1,yy,SP_init,0,1))
##println(f,"MASTER PROBLEM==========================================================================================")

zNum = 200000
cRefNum = 2000000
m = Model(() -> Gurobi.Optimizer(gurobi_env)) # If we want to add # in Gurobi, then we have to turn of Gurobi's own Cuts 
# println("1")
@variable(m, w)
@variable(m,  x[1:Len], Bin)
@variable(m, 1e6 >= z[1:zNum] >= 0)
# @constraintref constr[1:200000]
# @ConstrRef constr[1:200000]

# constr = Array{JuMP.JuMPArray{JuMP.ConstraintRef,1,Tuple{Array{Int64,1}}}}()

constr = Array{JuMP.ConstraintRef}(undef, cRefNum)
@constraint(m, sum(x[i] for i=1:Len) <= b) #attack budget = 2
constr[1] = @constraint(m, w <= SP_init + sum(yy[i]*x[i]*d[i] for i=1:Len) + z[1]  )
# @constraint(m,con[1], w <= SP_init + sum(yy[i]*x[i]*d[i] for i=1:Len) + z[1]  )
# @constraint(m,con[2], w >=0  )
#     println(sum(yy[i]*x[i]*d[i] for i=1:Len))
@objective(m, Max, w - (1/α)*sum(p[i]*z[i] for i = 1:length(p)) )#w - sum(s[k] for k=1:length(s))/length(s) )
# println(2)
##println(f,z[1:length(p)])

con_num = 1
stopping_cond = delta2 + 0
total_time = 0.0
iter = 0
x_count = 1
K_not = []
K_α = [1]
K_α_not = []
K_delta2 = [] #Separate cells passing O.C.2 from skipped cells
Freeze1_trigger = false
start = time()
terminate_cond = false

while terminate_cond == false || stopping_cond >= delta2 
# while length(K_α_not) > 0 || stopping_cond >= delta2 
#     println("WHILE")
    stopping_cond = 0
    iter = iter + 1
#         println(f,"\n\nITERATION ",length(df[:CELL]),": ========================================================================================== ")
#         println("\nIter : ", iter)

    optimize!(m) 
#     println("3")
    MP = JuMP.objective_value.(m)
    MP_obj = copy(MP)
      
#     println("4")
    x_now = JuMP.value.(x) #+ 0.0 
#     println("x_now = " , x_now)
    println("\nIter : ", iter)
    println("Obj = ", MP_obj)
    println("Interdiction ", findall(x_now.==1))
    println("# Cells = ", length(p))
    
    

        
        
#     if iter == 5
#         println()
#         println(p)
#         println(constr[1:50])
#         println(sort(df_Elim, [:gU,:gL]))
#     end
#     println("K_not = ", K_not)
#     println("K_delta2 = ", K_delta2)
#     println("K_not length = ", length(K_not))# , " out of ", length(p))
#     println("K_delta2 length = ", length(K_delta2), " out of ", length(p))
#     println("Cells No. = ", length(p))
#     println("g = ", g)
#     println("h = ", h)
#     println("p = ", p)
#     println(constr[1:con_num])
#     println("x_count = ", x_count)

    last_x = x_now
    z_now = JuMP.value.(z) #+ 0.0 
    k = 0
    myLength = length(p)
    
    if last_x != x_now
        K_not = [] #collect(1:myLength)
#         K_delta2 = []
        K_α = collect(1:myLength)
    end
    
#     println(minimum(p))
#=====#  
#     if minimum(p) <= 1-α
#         Freeze1_trigger = true
# #         println("Trigger set off.")
#     end
    
#     if Freeze1_trigger == true
#         println("x_count = ", x_count)
        
        
#     end
#=====#    
#     if terminate_cond == false
    zCutAdded = false
#     println("Before delta1")
    for k = 1:myLength 
        if (k in K_not) == false #&& (k in K_delta2) == false
#                 println(df)
#                 println("k = ",k)
            row = findall(df[!,:CELL] .== k)[end]
            c_L = df[!,:LB][row] 
            c_U = df[!,:UB][row] 
            c = (c_U + c_L)/2
            M = zeros(Len)
            for i = 1:Len
                M[i] = c_U[i] - c_L[i]
            end
            c_g = c + d.*x_now
#             println(k,". ", c)
            y, gx, SP = gx_bound(c_L, c_U, c, c_g, x_now, edge)
            g[k] = gx 
            df_gh[!,:g][k] = gx
            df_gh[!,:ARCS_USED][k] = y
            
            yL, gL = gx_bound(c_L, c_L, c_L, c_L+d.*x_now, x_now, edge)
            yU, gU = gx_bound(c_U, c_U, c_U, c_U+d.*x_now, x_now, edge)
            
            df_Elim[k,:gL] = gL
            df_Elim[k,:gU] = gU
#             df_Elim[k,:PROB] = gL
            
            if  JuMP.value.(w) - (z_now[k] + g[k]) >= epsilon
                push!(df, (k, y, c_L + zeros(length(c_L)), c_U + zeros(length(c_U)), p[k], SP))
                con_num = con_num + 1 
                constr[con_num] = @constraint(m, w <= sum(c[i]*y[i] + d[i]*y[i]*x[i] for i = 1:length(c_L)) + z[k])
                stopping_cond = delta2 + 1  
                if JuMP.value.(w) - (z_now[k] + g[k]) > delta1     
                    zCutAdded = true
                end
            end
        end
    end
    @objective(m, Max, w - (1/α)*sum(p[i]*z[i] for i = 1:length(p)) )
    ghCutAdded = false
#     println("zCutAdded = ", zCutAdded)
#     println("Before delta2")
#     println("K_not = ", K_not)
    if zCutAdded == false 
        if x_count >= 2#5
            K_not, K_α, K_α_not, terminate_cond = Freeze1(K_not)
#             println("K_α = ", K_α)
#             println("K_α_not = ", K_α_not)
#             println("K_not = ", K_not)
#             println("terminate_cond = ", terminate_cond)
            x_count = 1
        end
        x_count = x_count + 1
        for k = 1:myLength #length(p)# k = 1: cell_num #length(p) 
#             println("In 2nd big loop, k = ", k)
            if (k in K_not) == false #&& (k in K_delta2) == false #Commenting the K_delta2 set out
    #                 println(df[:CELL])
    #                     if length(find(df[:CELL] .== k))>0
                row = findall(df[!,:CELL] .== k)[1]
                c_L = df[!,:LB][row] 
                c_U = df[!,:UB][row]
#                 println("ARCS_USED = ",df[!,:ARCS_USED][row]  )
    #                     else
    #                         c_L = cL_orig[:,1] 
    #                         c_U = cU_orig[:,1] 
    #                     end
                c = (c_U + c_L)/2
                M = zeros(Len)
                for i = 1:Len
                    M[i] = c_U[i] - c_L[i]
                end
                c_g = c + d.*x_now

                ##println(f,"==================SUBPROBLEM 2: H(X), CURRENT CELL = ", k)
#                     if (k in K_delta2) == false
                y_h, hx = hx_bound(c_L, c_U, d, x_now)
                h[k] = hx
                df_gh[!,:h][k] = hx
#                     end
#                 println(k, ". g = ", g[k],"; h = ", h[k], " p = ", p[k])
#                 println("\tPre-partition:", SP)
#                 println("\ty = ", findall(df[!,:ARCS_USED][row].==1))
#                 println("g = ", g[k],"; h = ", h[k])
#                 println(k, ". g = ", g[k],"; h = ", h[k], " p = ", p[k])
#                 println("\t ", c_U[M.>0])
#                 println("\t ", c_L[M.>0])
                if g[k]-h[k] > epsilon
#                     println("Pass the first IF")
                    if g[k]-h[k] > delta2 #Commenting the delta2 condition out
#                         println("g - h > delta2")
    #                     println("Split cell ", k)
                        stopping_cond = delta2 + 1
                        found = true
                    end
                    #Prepping for chooseWorst function:
                    #Solve (3) at cL
                    if g[k] - h[k] > delta2 || k in K_α_not
#                         println("In g[k] - h[k] > delta2 || k in K_α_not")
                        
                        cg_k = c_L + d.*x_now
                        yL_k, gL_k = gx_bound(c_L, c_L, c_L, cg_k, x_now, edge)

                        #Find cW and solve (3) at cW
                        c_W = c_L + yL_k.*M

                        cgW_k = c_W + d.*x_now
                        yW_k, gW_k = gx_bound(c_W, c_W, c_W, cgW_k, x_now, edge)
                        
                        #Find arc
                        arc_split = 0
                        if yL_k != yW_k
                            arc_split = chooseWorst(M, yL_k, yW_k)#findall(M .== maximum(M))[1]
                        else
                            arc_split = chooseMax(M)
                        end
    #                     println("\tSplit ", arc_split)
                        #Split at middle per usual
                        gap = (c_U[arc_split] - c[arc_split])/2
                        newCell = maximum(df[!,:CELL])+1
#                         println("\ta. Partition ", k, " -> ", newCell)
                        
                        p_temp = p[k]/2
                        p[k] = p_temp
                        df_gh[!,:PROB][k] = p_temp
                        
                        push!(p,p_temp)
                        push!(K_α_not, newCell)
                        c_Mid_k = copy(c_U)
                        c_Mid_k[arc_split] = (c_U[arc_split]+c_L[arc_split])/2
                        c_Mid_newCell = copy(c_L)
                        c_Mid_newCell[arc_split] = c_Mid_k[arc_split]

                        c_Ltemp = (c_L + c_Mid_k)/2
                        c_g_Ltemp = c_Ltemp + d.*x_now

                        c_Utemp = (c_U + c_Mid_newCell)/2
                        c_g_Utemp = c_Utemp + d.*x_now

                        y_low, gx_low, SP_low = gx_bound(c_L, c_Mid_k, c_Ltemp, c_g_Ltemp, x_now, edge)
                        g[k] =  gx_low
                        df_gh[!,:g][k] = gx_low
                        df_gh[!,:ARCS_USED][k] = y_low
    #                     println("\tk_rev = ", gx_low,"; y = ", findall(y_low.==1))
                        
                        #Remember to Comment these - cL and cU - out since we have calculated this earlier
                        cg_k = c_L + d.*x_now
                        yL_k, gL_k = gx_bound(c_L, c_L, c_L, cg_k, x_now, edge)

                        cg_k = c_Mid_k + d.*x_now
                        yU_k, gU_k = gx_bound(c_Mid_k, c_Mid_k, c_Mid_k, cg_k, x_now, edge)
                        
                        
                        df_Elim[k,:gL] = gL_k
                        df_Elim[k,:gU] = gU_k
                        df_Elim[k,:PROB] = p_temp
    
                        y_high, gx_high, SP_high = gx_bound(c_Mid_newCell, c_U, c_Utemp, c_g_Utemp, x_now, edge)
                        push!(g, gx_high)
                        push!(h, 0)
#                         println("\t",k,"_rev = ", gx_low, "; ",newCell,". ", gx_high )
#                         println("\t","y_",k," = ",findall(y_low.==1),"y_",newCell," = ",findall(y_high.==1))
    #                     println("\tnewCell = ", gx_high,"; y = ", findall(y_high.==1))

                        cg_new = c_Mid_newCell + d.*x_now
                        yL_new, gL_new = gx_bound(c_Mid_newCell, c_Mid_newCell, c_Mid_newCell, cg_new, x_now, edge)
                        cg_new = c_U + d.*x_now
                        yU_new, gU_new = gx_bound(c_U, c_U, c_U, cg_new, x_now, edge)

                        push!(df_Elim, (newCell, gL_new, gU_new, p_temp))
                        push!(df_gh, (newCell,y_high, gx_high,0,p_temp))

                        y_low_exists = false
                        y_high_exists = false
#                         println("y_low_exists = ", y_low_exists)
                        constr_of_k = findall(df[!,:CELL].== k)

                        for i in constr_of_k #LOOP THRU ALL ROWS IN DF ASSOCIATED WITH k
                            df[!,:PROB][i] = p_temp #Probability must change
                            ARCS_USED_i = df[!,:ARCS_USED][i] #Get path y in that row/constraint
                            newCell_RHS = df[!,:SP][i] #Get a temporary RHS for newCell
                            df[!,:UB][i] = c_Mid_k
                            #ONLY UPDATE RHS IF ARC SPLIT IS ON THAT PATH           
                            if arc_split in findall(ARCS_USED_i.==1)
                                k_new_RHS = df[!,:SP][i] - gap #UPDATE RHS OF CELL k
                                newCell_RHS = df[!,:SP][i] + gap #UPDATE RHS OF NEWCELL
                                df[!,:SP][i] = k_new_RHS #FIX DF INFORMATION
        #                         JuMP.setRHS(constr[i], k_new_RHS)
                                set_normalized_rhs(constr[i], k_new_RHS)
                            end

                            #COPY THE CONSTRAINT FROM k to NEWCELL
                            con_num = con_num + 1
                            constr[con_num] = @constraint(m, w <= 
                                        sum(d[i]*x[i]*ARCS_USED_i[i] for i = 1:length(c_L)) + newCell_RHS + z[newCell])

                            push!(df, (newCell, ARCS_USED_i, c_Mid_newCell, c_U, p_temp, newCell_RHS))
                            
                            if ARCS_USED_i == y_low
                                y_low_exists = true
                            end
                            if ARCS_USED_i == y_high
                                y_high_exists = true
                            end
                        end
#                     end
                        if y_low_exists == false
                            push!(df, (k, y_low, c_L, c_Mid_k, p_temp, SP_low)) 
                            con_num = con_num + 1
                            constr[con_num] = @constraint(m, w <= 
                                        sum(d[i]*x[i]*y_low[i] for i = 1:length(c_L)) + SP_low + z[k])
                        end
                        if y_high_exists == false
                            push!(df, (newCell, y_high, c_Mid_newCell, c_U, p_temp, SP_high)) 
                            con_num = con_num + 1
                            constr[con_num] = @constraint(m, w <= 
                                        sum(d[i]*x[i]*y_high[i] for i = 1:length(c_L)) + SP_high + z[newCell])
                        end

                        ghCutAdded = true
                    end
                else
#                     println("Pass O2")
#                     println(K_α_not)
                    if (k in K_α_not) == true#df_Elim[!,:gU][k] < max_g
                        
                        y = df_gh[!,:ARCS_USED][k]
                        gx = g[k]
#                         df_gh[!,:g][k] = gx
                        gx = df_gh[!,:g][k]
                        
                        gL_k = df_Elim[!,:gL][k]
                        gU_k = df_Elim[!,:gU][k]

                        M_temp = y.*M
                        arc_split = findall(M_temp .== maximum(M_temp))[1]
#                         println("arc_split = ", arc_split)
#                         gap = (c_U[arc_split] - c[arc_split])/2

                        newCell = maximum(df[!,:CELL])+1
                        
                        p_temp = p[k]/2
                        p[k] = p_temp
                        df_gh[!,:PROB][k] = p_temp
                        push!(p,p_temp)

                        c_Mid_k = copy(c_U)
                        c_Mid_k[arc_split] = (c_U[arc_split]+c_L[arc_split])/2

                        c_Mid_newCell = copy(c_L)
                        c_Mid_newCell[arc_split] = c_Mid_k[arc_split]

                        #Update revised_cell_k:
                        df_Elim[!,:gU][k] = gU_k - M[arc_split]/2 #gap #gap = M[arc_split]/2
                        df_Elim[!,:PROB][k] = p_temp
                        gL_new = gL_k + M[arc_split]/2 #gap
                        push!(df_Elim, (newCell, gL_new, gU_k, p_temp))
                        

                        
                        g[k] = gx - M[arc_split]/4 #gap#/2
                        h[k] = g[k]
                        gx_high = gx + M[arc_split]/4# gap#/2
                        df_gh[!,:g][k] = g[k]#gx - gap/2
#                         df_gh[!,:g][k] = y
                        df_gh[!,:h][k] = g[k]#gx - gap/2
#                         df_gh[!,:g][k]
                        push!(g, gx_high)
                        push!(h, gx_high)
                        push!(df_gh, (newCell, y, gx_high, gx_high,p_temp))
                        constr_of_k = findall(df[!,:CELL].== k)
                        
#                         println("\tb. Partition ", k, " -> ", newCell, " on arc ", arc_split)
#                         println("\t",k,"_rev = ", g[k], "; ",newCell,". ", gx_high )
#                         println("\t","y_",k," = ",findall(y.==1),"y_",newCell," = ",findall(y.==1))
#                         if k == 3
#                             println("\tb. Partition ", k, " -> ", newCell)
#                             println("g_k = ", g)
#                         end
                        for i in constr_of_k #LOOP THRU ALL ROWS IN DF ASSOCIATED WITH k
                            df[!,:PROB][i] = p_temp #Probability must change
                            ARCS_USED_i = df[!,:ARCS_USED][i] #Get path y in that row/constraint
                            newCell_RHS = df[!,:SP][i] #Get a temporary RHS for newCell
                            df[!,:UB][i] = c_Mid_k
                            #ONLY UPDATE RHS IF ARC SPLIT IS ON THAT PATH           
                            if arc_split in findall(ARCS_USED_i.==1)
                                k_new_RHS = df[!,:SP][i] - M[arc_split]/4 #gap #UPDATE RHS OF CELL k
                                newCell_RHS = df[!,:SP][i] + M[arc_split]/4 #gap #UPDATE RHS OF NEWCELL
                                df[!,:SP][i] = k_new_RHS #FIX DF INFORMATION
        #                         JuMP.setRHS(constr[i], k_new_RHS)
                                set_normalized_rhs(constr[i], k_new_RHS)
                            end

                            #COPY THE CONSTRAINT FROM k to NEWCELL
                            con_num = con_num + 1
                            constr[con_num] = @constraint(m, w <= 
                                        sum(d[i]*x[i]*ARCS_USED_i[i] for i = 1:length(c_L)) + newCell_RHS + z[newCell])
                            push!(df, (newCell, ARCS_USED_i, c_Mid_newCell, c_U, p_temp, newCell_RHS))
                        end

                         #CAREFUL THIS IS NOT(!!) NOREP
#                         else
#                             push!(K_delta2,k)
                    end
                end #END OF if gx-hx > delta2
            end
        end #END OF for k = 1:myLength
#         println("Out of 2nd big loop")
        @objective(m, Max, w - (1/α)*sum(p[i]*z[i] for i = 1:length(p)))
    end
#     if length(p) > 3
#         temp_df_gh = sort(df_gh,[:g,:h])
#     #     ii = Int(ceil(α*length(p),digits=0))
# #         println(df)
# #         println(df_gh)
# #         println("####")
# #         println(temp_df_gh)
#         ii=1
#         myP = temp_df_gh[!,:PROB][ii]
#         while myP < α
#             ii=ii+1
#             myP = myP + temp_df_gh[!,:PROB][ii]
#         end
        
#         mySum = sum(temp_df_gh[!,:PROB][i] for i = 1:(ii-1))
# #         println("mySum = ",mySum)
# #         println("1/K = ", 1/length(p))
# #         println("g_avg = ", sum(temp_df_gh[!,:g][i]*temp_df_gh[!,:PROB][i] for i = 1:(ii-1))+(α-mySum)*temp_df_gh[!,:g][ii])
# #         println("h_avg = ", sum(temp_df_gh[!,:h][i]*temp_df_gh[!,:PROB][i] for i = 1:(ii-1))+(α-mySum)*temp_df_gh[!,:h][ii])
#     end
#     println("g = ", g)
#     println("h = ", h)
    elapsed = time() - start
#     end
#     time_lapse = toq()    
#     total_time = total_time + time_lapse
#         if total_time > 3600
#             total_time = ">3600"
# #             println("total time" > 3600")
#             break
#         end
end
    
println(df)
println(df_gh)
total_time = time() - start

@timeit to "CALC OPT GAP" opt_gap = sum(p[i]*(z[i] - h[i]) for i = 1:length(p))
##println(f,"\n\n\nFinal z = ", getvalue(z[1:length(p)]))
# println(f,"Final g = ", g)
# println(f,"Final h = ",h)
##println(f,"List of all cells = ", Cell_List)

println("\n\nInstance ", myFile)
println("α = ", α)
println("delta2 = ", delta2)
println("Interdiction = ", findall(last_x.==1))
println("MP_obj = ", MP_obj)
println("K_not length = ", length(K_not), " out of ", length(p))
if isempty(K_not)
    println("K_not weight total = ", 0)
else
    println("K_not weight total = ", sum(p[k] for k in K_not))
end
println("Overall runtime = ", total_time)
println("No. iterations = ", iter) #length(df[:CELL]))


Iter : 1
Obj = 122.0
Interdiction [1]
# Cells = 1

Iter : 2
Obj = 110.0
Interdiction [1, 8]
# Cells = 1

Iter : 3
Obj = 22.0
Interdiction [2, 8]
# Cells = 1

Iter : 4
Obj = 22.0
Interdiction [2, 8]
# Cells = 1

Iter : 5
Obj = 13.0
Interdiction [1, 8]
# Cells = 2

Iter : 6
Obj = 13.0
Interdiction [1, 8]
# Cells = 4

Iter : 7
Obj = 12.0
Interdiction [2, 8]
# Cells = 6

Iter : 8
Obj = 10.0
Interdiction [1, 2]
# Cells = 8

Iter : 9
Obj = 9.765625
Interdiction [1, 2]
# Cells = 9

Iter : 10
Obj = 9.53125
Interdiction [1, 2]
# Cells = 11

Iter : 11
Obj = 9.296875
Interdiction [1, 2]
# Cells = 14

Iter : 12
Obj = 9.296875
Interdiction [1, 2]
# Cells = 17

Iter : 13
Obj = 9.0625
Interdiction [1, 2]
# Cells = 21

Iter : 14
Obj = 9.0625
Interdiction [1, 2]
# Cells = 24

Iter : 15
Obj = 9.0625
Interdiction [1, 2]
# Cells = 28

Iter : 16
Obj = 8.875
Interdiction [2, 8]
# Cells = 39

Iter : 17
Obj = 8.5
Interdiction [1, 8]
# Cells = 52

Iter : 18
Obj = 8.5
Interdiction [1, 2]
# Cells = 79

Iter : 1

 900 │   263  [0, 1, 0, 0, 1, 0, 1, 0, 0]        [2.0, 1.0, 30.0, 0.0, 0.0, 12.0,…  [2.0, 1.0, 40.0, 0.0, 0.0, 12.0,…  0.000488281   4.75
263×5 DataFrame
 Row │ CELL   ARCS_USED                    g         h         PROB        
     │ Int64  Array                        Float64   Float64   Float64     
─────┼─────────────────────────────────────────────────────────────────────
   1 │     1  [0, 0, 0, 0, 0, 0, 0, 1, 1]   7.375     7.375    0.00390625
   2 │     2  [0, 0, 0, 0, 0, 0, 0, 1, 1]   7.375     7.375    0.00390625
   3 │     3  [0, 0, 0, 0, 0, 0, 0, 1, 1]   7.375     7.375    0.0078125
   4 │     4  [0, 0, 0, 0, 0, 0, 0, 1, 1]   7.375     7.375    0.0078125
   5 │     5  [0, 0, 0, 0, 0, 0, 0, 1, 1]   7.375     7.375    0.0078125
   6 │     6  [0, 0, 0, 0, 0, 0, 0, 1, 1]   7.375     7.375    0.00390625
   7 │     7  [0, 0, 0, 0, 0, 0, 0, 1, 1]   7.375     7.375    0.00195313
   8 │     8  [0, 0, 0, 0, 0, 0, 0, 1, 1]   7.375     7.375    0.00390625
   9 │     9  [0, 0, 0, 0, 0,



Instance ./myData1.jl
α = 0.2
delta2 = 1.0
Interdiction = [1, 2]
MP_obj = 7.60443115234375
K_not length = 0 out of 263
K_not weight total = 0
Overall runtime = 22.721999883651733
No. iterations = 29


In [124]:
println(elapsed)
# println(findall(last_x.==1))
a = [1,3,2]
b = [8,7,6]
c = [8,2,4]
# Pkg.add("Statistics")
# using Statistics

df_cells = DataFrame(a = Float64[],b = Float64[], c = Float64[])
for i=1:3
    push!(df_cells, (a[i],b[i],c[i]))
end

println(df_cells)
# sort(df_cells, (:c,:b))
# print(mean(df_cells.a))
# temp = Array{Float64,2}
# temp = hcat(a,b,c)
# print(temp)
# sort(temp)
# print(temp)
# n = [1,3]
# println(sum(a[i] for i in n))
# i=3
# myRows= collect(i:nrow(df_cells))
# sort(df_cells,[:b,:a])
# df_cells = filter!(row -> (row.a >1), df_cells[myRows, :])
# row = 3
# start = time()
# for i = 1:10000000
#     df_cells[row, :c]
# end
# println(time()-start)
# start = time()
# for i = 1:10000000
#     df_cells[!,:c][row]
# end
# println(time()-start)
# filter(row -> row.gL >= max_g, temp_df[myRows,:])
df_cells = filter(row -> row.a > 1, df_cells[[2,3],:])[!,:a]
# df_cells = df_cells[[2,3],:a]#[2,3]

UndefVarError: UndefVarError: elapsed not defined

In [ ]:
Lim1 = Array{ConstraintRef}, 1:2000

In [ ]:
array1 = Int[]

In [ ]:
array2 = ConstraintRef[]

In [ ]:
testModel = Model(() -> Gurobi.Optimizer(gurobi_env)) # If we want to add # in Gurobi, then we have to turn of Gurobi's own Cuts 
# println("1")
# @variable(testModel, w)


@variable(testModel,  x[1:2]>=0)
# @variable(m, 1e6 >= z[1:200000] >= 0)

@constraint(testModel, x[1]+x[2] >= 5)
@constraint(testModel, con, 2*x[1] - 3x[2] <= 2.5)
set_normalized_rhs(con,13)
@objective(testModel,Min, x[1] + 2*x[2])

println(testModel)
# optimize!(testModel)
# println(JuMP.value.(x))

In [ ]:
# testModel.update()
test2 = copy(testModel)
x_new = copy(x,test2)

@constraint(test2, x_new[1] >= 3)
@objective(test2, Min, 2*x_new[2])
# test2.update()
set_optimizer(test2, ()-> Gurobi.Optimizer(gurobi_env))
println(test2)
optimize!(test2)


In [ ]:
println(findall([1, 0, 1].== 1))

In [ ]:
list = [1, 2.5, 3.5, 7]

In [ ]:
insert_and_dedup!(v::Vector, x) = (splice!(v, searchsorted(v,x), [x]); v)

In [ ]:
insert_and_dedup!(list, 3.5)

In [ ]:
import Pkg
Pkg.add("DataStructures")
using DataStructures


In [ ]:
s = SortedSet{Float64}()
SortedSet(Float64[],
Base.Order.ForwardOrdering())

In [ ]:
push!(s,10)

In [ ]:
s = [1,2,2,2,3,3,3]

println(findall(s .== 3))

In [ ]:
#Sort Testing
# Pkg.add("Random")
using Random
df_cell = DataFrame(CELL = Int[], gU = Float64[], PROB = Float64[])
push!(df_cell, (1, 100, 1))
# cell = [1]
# p = 
println(df_cell)
newCell = 1
start = time()
temp_df = DataFrame(CELL = Int[], gU = Float64[], PROB = Float64[])
for i = 1:6
#     println(df_cell[!,:CELL])
    for k = 1:newCell #in df_cell[!,:CELL]
        if rand() >0.5
            newCell = newCell + 1
            p_k = df_cell[k,:PROB]
            gU_k = df_cell[k,:gU]
            r = gU_k+5
    #         println(gU_k)
    #         println(r)
            gU_new = rand(gU_k:r)
    #         println(gU_new)
            r = gU_k-5
            gU_k = rand(r:gU_k)
            df_cell[k,:PROB] = p_k/2
            df_cell[k,:gU] = gU_k
            p_new = p_k/2
            push!(df_cell, (newCell, gU_new, p_new))
        end
    end
    temp_df = sort!(df_cell, [:gU,:PROB])
#     println(df_cell)
end
elapsed = time() - start
println(elapsed)
# println(df_cell)
println(temp_df)
# df_cell
# temp_df[temp_df.gU >100]
temp_df = filter(row -> row.gU > 100, df_cell)
# println(temp_df[!,:CELL])

In [ ]:
println(df_cell)

In [ ]:
a = [1,2,3,4]
# a[1],a[4] = a[4],a[1]
# pop!(a)
a

In [ ]:
println(a)
for i in a
    println(i)
    delete!(a, 2)
#     a[i],a[end] = a[end],a[i]
#     pop!(a)
    println(a)
end

In [ ]:
a = [1,2,3,4,5]
a = rand(5)
println(a)
filter(x->x.>2,a)
println(a)

In [ ]:
println(a)
sort!(a)
println(a)

In [ ]:
a = 2.5555
println(Int(ceil(a,digits=0)))